## Imports

In [48]:
%pip install kagglehub
%pip install pandas
%pip install tabulate
%pip install spacy
%pip install torch
%pip install dask
%pip install "dask[dataframe]"
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -------------------------------- ------ 10.7/12.8 MB 56.1 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 53.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [49]:
import kagglehub
import pandas as pd
import os
import json
from tabulate import tabulate
import re
import spacy
from spacy.matcher import Matcher
import torch
import dask
import dask.dataframe as dd


In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device.upper()}")


Running on: CUDA


## Preprocessing of O*NET Database

### Loading Dataset

In [51]:
onet_path = r'ONET Dataset\db_29_2_text\db_29_2_text'

In [52]:
onet_occupation_df = pd.read_csv(onet_path + '\Occupation Data.txt', delimiter='\t')
onet_skills_df = pd.read_csv(onet_path + '\Skills.txt', delimiter='\t')
onet_knowledge_df = pd.read_csv(onet_path + '\Knowledge.txt', delimiter='\t')
onet_abilities_df = pd.read_csv(onet_path + '\Abilities.txt', delimiter='\t')
onet_work_activities_df = pd.read_csv(onet_path + '\Work Activities.txt', delimiter='\t')

print("Occupation: ", onet_occupation_df.head())
print("Skills: ", onet_skills_df.head())
print("Knowledge: ", onet_knowledge_df.head())
print("Abilities: ", onet_abilities_df.head())
print("Work Activities: ", onet_abilities_df.head())

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\K'
<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\K'
<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:5: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Keerthana\AppData\Local\Temp\ipykernel_8936\3415243021.py:1: SyntaxWarning: invalid escape sequence '\O'
  onet_occupation_df = pd.read_csv(onet_path + '\Occupation Data.txt', delimiter='\t')
C:\Users\Keerthana\AppData\Local\Temp\ipykernel_8936\3415243021.py:2: SyntaxWarning: invalid escape sequence '\S'
  onet_skills_df = pd.read_csv(onet_path + '\Skills.txt', delimiter='\t')
C:\Users\Keerthana\AppData\Local\Temp\ipykernel_8936\3415243021.py:3: SyntaxWarning: invalid escape sequence '\

Occupation:    O*NET-SOC Code                                Title  \
0     11-1011.00                     Chief Executives   
1     11-1011.03        Chief Sustainability Officers   
2     11-1021.00      General and Operations Managers   
3     11-1031.00                          Legislators   
4     11-2011.00  Advertising and Promotions Managers   

                                         Description  
0  Determine and formulate policies and provide o...  
1  Communicate and coordinate with management, sh...  
2  Plan, direct, or coordinate the operations of ...  
3  Develop, introduce, or enact laws and statutes...  
4  Plan, direct, or coordinate advertising polici...  
Skills:    O*NET-SOC Code Element ID           Element Name Scale ID  Data Value  N  \
0     11-1011.00    2.A.1.a  Reading Comprehension       IM        4.12  8   
1     11-1011.00    2.A.1.a  Reading Comprehension       LV        4.62  8   
2     11-1011.00    2.A.1.b       Active Listening       IM        4.00

### Filtering Skills.txt

In [53]:
onet_skills_filtered = onet_skills_df[(onet_skills_df['Scale ID'] == 'IM') & (onet_skills_df['Data Value'] > 0)]
onet_skills_filtered = onet_skills_filtered[['O*NET-SOC Code', 'Element Name', 'Data Value']]
onet_skills_filtered = onet_skills_filtered.rename(columns={'Element Name': 'Skill', 'Data Value': 'Importance'})

print(onet_skills_filtered.head())

  O*NET-SOC Code                  Skill  Importance
0     11-1011.00  Reading Comprehension        4.12
2     11-1011.00       Active Listening        4.00
4     11-1011.00                Writing        4.12
6     11-1011.00               Speaking        4.25
8     11-1011.00            Mathematics        3.25


### Filtering Knowledge.txt

In [54]:
onet_knowledge_filtered = onet_knowledge_df[(onet_knowledge_df['Scale ID'] == 'IM') & (onet_knowledge_df['Data Value'] > 0)]
onet_knowledge_filtered = onet_knowledge_filtered[['O*NET-SOC Code', 'Element Name', 'Data Value']]
onet_knowledge_filtered = onet_knowledge_filtered.rename(columns={'Element Name': 'Knowledge', 'Data Value': 'Importance'})

print(onet_knowledge_filtered.head())

  O*NET-SOC Code                      Knowledge  Importance
0     11-1011.00  Administration and Management        4.78
2     11-1011.00                 Administrative        2.42
4     11-1011.00       Economics and Accounting        4.04
6     11-1011.00            Sales and Marketing        3.81
8     11-1011.00  Customer and Personal Service        4.39


### Filtering Abilities.txt

In [55]:
onet_abilities_filtered = onet_abilities_df[(onet_abilities_df['Scale ID'] == 'IM') & (onet_abilities_df['Data Value'] > 0)]
onet_abilities_filtered = onet_abilities_filtered[['O*NET-SOC Code', 'Element Name', 'Data Value']]
onet_abilities_filtered = onet_abilities_filtered.rename(columns={'Element Name': 'Ability', 'Data Value': 'Importance'})

print(onet_abilities_filtered.head())

  O*NET-SOC Code                Ability  Importance
0     11-1011.00     Oral Comprehension        4.62
2     11-1011.00  Written Comprehension        4.25
4     11-1011.00        Oral Expression        4.50
6     11-1011.00     Written Expression        4.12
8     11-1011.00       Fluency of Ideas        3.88


### Filtering Work Activities.txt

In [56]:
onet_work_activities_filtered = onet_work_activities_df[(onet_work_activities_df['Scale ID'] == 'IM') & (onet_work_activities_df['Data Value'] > 0)]
onet_work_activities_filtered = onet_work_activities_filtered[['O*NET-SOC Code', 'Element Name', 'Data Value']]
onet_work_activities_filtered = onet_work_activities_filtered.rename(columns={'Element Name': 'Ability', 'Data Value': 'Importance'})

print(onet_work_activities_filtered.head())

  O*NET-SOC Code                                            Ability  \
0     11-1011.00                                Getting Information   
2     11-1011.00   Monitoring Processes, Materials, or Surroundings   
4     11-1011.00           Identifying Objects, Actions, and Events   
6     11-1011.00     Inspecting Equipment, Structures, or Materials   
8     11-1011.00  Estimating the Quantifiable Characteristics of...   

   Importance  
0        4.56  
2        4.25  
4        4.23  
6        2.51  
8        3.26  


### Merging Skills, Knowledge and Abilities to Occupation

In [57]:
onet_occupation_skills = onet_occupation_df[['O*NET-SOC Code', 'Title']].merge(onet_skills_filtered, on='O*NET-SOC Code', how='left')
onet_occupation_knowledge = onet_occupation_df[['O*NET-SOC Code', 'Title']].merge(onet_knowledge_filtered, on='O*NET-SOC Code', how='left')
onet_occupation_abilities = onet_occupation_df[['O*NET-SOC Code', 'Title']].merge(onet_abilities_filtered, on='O*NET-SOC Code', how='left')
onet_occupation_work_activities = onet_occupation_df[['O*NET-SOC Code', 'Title']].merge(onet_work_activities_filtered, on='O*NET-SOC Code', how='left')

print(onet_occupation_skills.head())
print(onet_occupation_knowledge.head())
print(onet_occupation_abilities.head())
print(onet_occupation_work_activities.head())

  O*NET-SOC Code             Title                  Skill  Importance
0     11-1011.00  Chief Executives  Reading Comprehension        4.12
1     11-1011.00  Chief Executives       Active Listening        4.00
2     11-1011.00  Chief Executives                Writing        4.12
3     11-1011.00  Chief Executives               Speaking        4.25
4     11-1011.00  Chief Executives            Mathematics        3.25
  O*NET-SOC Code             Title                      Knowledge  Importance
0     11-1011.00  Chief Executives  Administration and Management        4.78
1     11-1011.00  Chief Executives                 Administrative        2.42
2     11-1011.00  Chief Executives       Economics and Accounting        4.04
3     11-1011.00  Chief Executives            Sales and Marketing        3.81
4     11-1011.00  Chief Executives  Customer and Personal Service        4.39
  O*NET-SOC Code             Title                Ability  Importance
0     11-1011.00  Chief Executives     Ora

### Cleaning Entries

In [58]:
def onet_clean_entry(entry):
  if isinstance(entry, str):
    entry = re.sub(r'<[^>]*>', '', entry)
    entry = re.sub(r'[^a-zA-Z0-9\s]', '', entry)
    entry = re.sub(r'\\s+', '', entry)
    return entry.lower().strip()
  else:
    return ''

onet_occupation_skills['Title'] = onet_occupation_skills['Title'].apply(onet_clean_entry)
onet_occupation_skills['Skill'] = onet_occupation_skills['Skill'].apply(onet_clean_entry)

onet_occupation_knowledge['Title'] = onet_occupation_knowledge['Title'].apply(onet_clean_entry)
onet_occupation_knowledge['Knowledge'] = onet_occupation_knowledge['Knowledge'].apply(onet_clean_entry)

onet_occupation_abilities['Title'] = onet_occupation_abilities['Title'].apply(onet_clean_entry)
onet_occupation_abilities['Ability'] = onet_occupation_abilities['Ability'].apply(onet_clean_entry)

onet_occupation_work_activities['Title'] = onet_occupation_work_activities['Title'].apply(onet_clean_entry)
onet_occupation_work_activities['Ability'] = onet_occupation_work_activities['Ability'].apply(onet_clean_entry)

### Uploading Processed Entries to CSV

In [59]:
onet_occupation_skills.to_csv('onet_occupation_skills.csv', index=False)
onet_occupation_knowledge.to_csv('onet_occupation_knowledge.csv', index=False)
onet_occupation_abilities.to_csv('onet_occupation_abilities.csv', index=False)
onet_occupation_work_activities.to_csv('onet_occupation_work_activities.csv', index=False)

print("Done")
print(onet_occupation_skills.head())
print(onet_occupation_knowledge.head())
print(onet_occupation_abilities.head())
print(onet_occupation_work_activities.head())

Done
  O*NET-SOC Code             Title                  Skill  Importance
0     11-1011.00  chief executives  reading comprehension        4.12
1     11-1011.00  chief executives       active listening        4.00
2     11-1011.00  chief executives                writing        4.12
3     11-1011.00  chief executives               speaking        4.25
4     11-1011.00  chief executives            mathematics        3.25
  O*NET-SOC Code             Title                      Knowledge  Importance
0     11-1011.00  chief executives  administration and management        4.78
1     11-1011.00  chief executives                 administrative        2.42
2     11-1011.00  chief executives       economics and accounting        4.04
3     11-1011.00  chief executives            sales and marketing        3.81
4     11-1011.00  chief executives  customer and personal service        4.39
  O*NET-SOC Code             Title                Ability  Importance
0     11-1011.00  chief executives   

## Preprocessing of CareerBuilder 2020 Database

### Loading Dataset

In [60]:
# Download latest version
careerbuilder_path = kagglehub.dataset_download("promptcloud/careerbuilder-job-listing-2020")

print("Path to dataset files:", careerbuilder_path)


# # Assuming the dataset is in CSV format, locate the file
# # List files in the downloaded path
# files = os.listdir(path)
# print("Files in the dataset directory:", files)

print(careerbuilder_path)

# df = pd.read_json(files, lines=True)
# print(df.head())

Path to dataset files: C:\Users\Keerthana\.cache\kagglehub\datasets\promptcloud\careerbuilder-job-listing-2020\versions\1
C:\Users\Keerthana\.cache\kagglehub\datasets\promptcloud\careerbuilder-job-listing-2020\versions\1


In [61]:
# for dirname, _, filenames in os.walk('/root/.cache/kagglehub'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk(careerbuilder_path):
    for filename in filenames:
        if filename.endswith(".ldjson"):
            ldjson_file_path = os.path.join(dirname, filename)
            break
print("ldjson file path:", ldjson_file_path)

ldjson file path: C:\Users\Keerthana\.cache\kagglehub\datasets\promptcloud\careerbuilder-job-listing-2020\versions\1\marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson


In [62]:
ldjson_file_path = r'CareerBuilder 2020 Dataset\archive\marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson'

# Read the LDJSON file line by line with specified encoding
data = []
with open(ldjson_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            line = line.strip()  # Remove leading and trailing whitespace
            if line:  # Ensure the line is not empty
                data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip lines that cannot be decoded

# Convert the list of JSON objects into a pandas DataFrame
careerbuilder_df = pd.DataFrame(data)

careerbuilder_df.head()
#df = df.drop_duplicates().dropna(subset=["Job Title", "Job Description"])

Error decoding JSON: Extra data: line 1 column 7004 (char 7003)


,uniq_id,crawl_timestamp,url,job_title,company_name,city,state,country,post_date,job_description,...,inferred_state,inferred_country,fitness_score,inferred_salary_from,inferred_salary_to,inferred_salary_currency,is_consumed_job,job_requirements,contact_email,test_contact_email
0,9a457ef257fecf231693a6ba08f50293,2020-06-26 01:54:03 +0000,https://www.careerbuilder.com/job/J3W0PF650Z8Q...,Asphalt/Concrete Senior Project Manager,GPAC,Houston,TX,US,2020-06-25,SR. PROJECT MANAGER WANTED!!! My client is a M...,...,Texas,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ba471e2faf6f79caf22cddebbedbc0e8,2020-05-17 01:21:05 +0000,https://www.careerbuilder.com/job/J3P0FL6X795R...,Amazon Warehouse Team - Full Time,Amazon Fulfillment,Cincinnati,OH,US,2020-05-16,"Shifts: Overnight, Early Morning, Day, Evening...",...,Ohio,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6f00bd02d63c633b5af453366f25c21e,2020-06-27 04:53:42 +0000,https://www.careerbuilder.com/job/J3W3XK69NHHR...,Amazon Warehouse Associate - Morning Shifts Av...,Amazon Fulfillment,Peabody,MA,US,2020-06-26,"Shifts: Early Morning, Day, Evening Location: ...",...,Massachusetts,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8ad0d00bfa23cfd7b7c364b8ae72085f,2020-06-03 01:21:32 +0000,https://www.careerbuilder.com/job/J3Q5RL6VS7PG...,Assembly Electrical,Manpower,Villard,MN,US,2020-06-02,Do you pride yourself on attention to detail a...,...,Minnesota,United states,10,14.00,14.00,$,NaN,NaN,NaN,NaN
4,31753dc342a1b2a07db712454c0d5f87,2020-05-23 01:19:07 +0000,https://www.careerbuilder.com/job/J3V1TJ68DPJ7...,Graphics Designer,The North West Company - U.S.,Anchorage,AK,US,2020-05-22,In your role as Graphics Designer for Alaska C...,...,Alaska,United states,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Extracting the Important Columns

In [63]:
careerbuilder_df = careerbuilder_df[['uniq_id', 'job_title', 'job_description', 'job_requirements']]
careerbuilder_df = careerbuilder_df.dropna(subset=["job_title", "job_description"])

### Cleaning Entries

In [64]:
text_processor = spacy.load('en_core_web_sm') #https://spacy.io/models/en#en_core_web_sm

def careerbuilder_clean_entry(entry):
  if isinstance(entry, str):
    entry = re.sub(r'<[^>]*>', '', entry)
    entry = re.sub(r'[^a-zA-Z0-9\s]', '', entry)
    entry = re.sub(r'\\s+', '', entry)
    return entry.lower().strip()
  else:
    return ''

careerbuilder_df['job_title_cleaned'] = careerbuilder_df['job_title'].apply(careerbuilder_clean_entry)
careerbuilder_df['job_description_cleaned'] = careerbuilder_df['job_description'].apply(careerbuilder_clean_entry)

print(careerbuilder_df['job_title_cleaned'])
print(careerbuilder_df['job_description_cleaned'])



0                   asphaltconcrete senior project manager
1                         amazon warehouse team  full time
2        amazon warehouse associate  morning shifts ava...
3                                      assembly electrical
4                                        graphics designer
                               ...                        
29977           part time or temporary warehouse associate
29978                            patient access specialist
29979              registered nurse  icu  13 week contract
29980    amazon prime now shopper  earn 15hr or more in...
29981                warehouse operator  earn 15hr or more
Name: job_title_cleaned, Length: 29982, dtype: object
0        sr project manager wanted my client is a midsi...
1        shifts overnight early morning day evening wee...
2        shifts early morning day evening location stou...
3        do you pride yourself on attention to detail a...
4        in your role as graphics designer for alaska c...
  

### Extracting Skills from Job Descriptions

In [65]:
onettocareer_skills_df = pd.read_csv('onet_occupation_skills.csv')
onettocareer_knowledge_df = pd.read_csv('onet_occupation_knowledge.csv')
onettocareer_abilities_df = pd.read_csv('onet_occupation_abilities.csv')
onettocareer_work_activities_df = pd.read_csv('onet_occupation_work_activities.csv')

In [66]:
skills_list = onettocareer_skills_df['Skill'].tolist()
knowledge_list = onettocareer_knowledge_df['Knowledge'].tolist()
abilities_list = onettocareer_abilities_df['Ability'].tolist()
work_activities_list = onettocareer_work_activities_df['Ability'].tolist()

print("Skills List: ", skills_list)
print("Knowledge List: ", knowledge_list)
print("Abilities List: ", abilities_list)
print("Work Activities List: ", work_activities_list)

Skills List:  ['reading comprehension', 'active listening', 'writing', 'speaking', 'mathematics', 'science', 'critical thinking', 'active learning', 'learning strategies', 'monitoring', 'social perceptiveness', 'coordination', 'persuasion', 'negotiation', 'instructing', 'service orientation', 'complex problem solving', 'operations analysis', 'technology design', 'equipment selection', 'installation', 'programming', 'operations monitoring', 'operation and control', 'equipment maintenance', 'troubleshooting', 'repairing', 'quality control analysis', 'judgment and decision making', 'systems analysis', 'systems evaluation', 'time management', 'management of financial resources', 'management of material resources', 'management of personnel resources', 'reading comprehension', 'active listening', 'writing', 'speaking', 'mathematics', 'science', 'critical thinking', 'active learning', 'learning strategies', 'monitoring', 'social perceptiveness', 'coordination', 'persuasion', 'negotiation', 'i

### Matching Skills, Knowledge and Abilities from O*NET with CareerBuilder

In [67]:
skill_matcher = Matcher(text_processor.vocab)
knowledge_matcher = Matcher(text_processor.vocab)
ability_matcher = Matcher(text_processor.vocab)
work_activities_matcher = Matcher(text_processor.vocab)

skill_patterns = [[{'LOWER': token.text} for token in text_processor.make_doc(str(skill))] for skill in skills_list]
knowledge_patterns = [[{'LOWER': token.text} for token in text_processor.make_doc(str(knowledge))] for knowledge in knowledge_list]
ability_patterns = [[{'LOWER': token.text} for token in text_processor.make_doc(str(ability))] for ability in abilities_list]
work_activities_patterns = [[{'LOWER': token.text} for token in text_processor.make_doc(str(work_activities))] for work_activities in work_activities_list]

skill_matcher.add("Skills", skill_patterns)
knowledge_matcher.add("Knowledge", knowledge_patterns)
ability_matcher.add("Abilities", ability_patterns)
work_activities_matcher.add("Work Activities", work_activities_patterns)

In [68]:
def extract_matches(entry, matcher):
    doc = text_processor(entry)
    matches = matcher(doc)
    return [doc[start:end].text for match_id, start, end in matches]

In [69]:
careerbuilder_df['extracted_skills'] = careerbuilder_df['job_description_cleaned'].apply(lambda x: extract_matches(x, skill_matcher))
careerbuilder_df['extracted_knowledge'] = careerbuilder_df['job_description_cleaned'].apply(lambda x: extract_matches(x, knowledge_matcher))
careerbuilder_df['extracted_abilities'] = careerbuilder_df['job_description_cleaned'].apply(lambda x: extract_matches(x, ability_matcher))
careerbuilder_df['extracted_work_activities'] = careerbuilder_df['job_description_cleaned'].apply(lambda x: extract_matches(x, work_activities_matcher))

print(careerbuilder_df['extracted_skills'])
print(careerbuilder_df['extracted_knowledge'])
print(careerbuilder_df['extracted_abilities'])
print(careerbuilder_df['extracted_work_activities'])

0        []
1        []
2        []
3        []
4        []
         ..
29977    []
29978    []
29979    []
29980    []
29981    []
Name: extracted_skills, Length: 29982, dtype: object
0                                                       []
1                                                       []
2                                                       []
3                                                       []
4        [design, design, design, design, design, desig...
                               ...                        
29977    [transportation, transportation, transportatio...
29978     [administrative, administrative, administrative]
29979                                                   []
29980                                                   []
29981                                                   []
Name: extracted_knowledge, Length: 29982, dtype: object
0        []
1        []
2        []
3        []
4        []
         ..
29977    []
29978    []
29979    []
29

In [70]:
careerbuilder_df.to_csv('careerbuilder_preprocessed.csv', index=False)
print("Careerbuilder Dataset has been preprocessed and saved to careerbuilder_preprocessed.csv")

Careerbuilder Dataset has been preprocessed and saved to careerbuilder_preprocessed.csv
